In [60]:
import pandas as pd
from datetime import datetime

import seaborn as sns

- Number of courses enrolled in: This could indicate the student's level of commitment and interest.
- Last active time: Indicates how recently the student engaged with the platform, potentially reflecting their current interest.
- Demographic Information: Age, gender, location: Depending on privacy concerns and available data, these attributes might provide insights into user behavior.

In [61]:
student_info_df = pd.read_csv('data/raw/365_student_info.csv')
course_ratings_df = pd.read_csv('data/raw/365_course_ratings.csv')
student_engagement_df = pd.read_csv('data/raw/365_student_engagement.csv')
student_purchases_df = pd.read_csv('data/raw/365_student_purchases.csv')

ml_dataset = pd.read_csv('data/processed/ml_dataset.csv')

In [62]:
student_engagement_df.date_engaged = pd.to_datetime(student_engagement_df.date_engaged)
student_info_df.date_registered = pd.to_datetime(student_info_df.date_registered)
student_purchases_df.date_purchased = pd.to_datetime(student_purchases_df.date_purchased)

In [63]:
df = pd.merge(student_info_df[['student_id', 'date_registered']],
              student_engagement_df[['student_id', 'date_engaged']],
              on='student_id',
              how='left')

df = df.drop_duplicates(subset='student_id', keep='last')

df['latest_date_recorded'] = datetime(2022, 10, 20).date()
df.latest_date_recorded = pd.to_datetime(df.latest_date_recorded)

df['last_active'] = (df.latest_date_recorded - df.date_engaged).dt.days

df.last_active = df.last_active.fillna((df.latest_date_recorded - df.date_registered).dt.days)
df = df[['student_id', 'last_active']]

df

,student_id,last_active
0,258798,292.0
1,258799,292.0
30,258800,114.0
31,258801,292.0
32,258802,292.0
...,...,...
82252,295511,0.0
82253,295512,0.0
82254,295513,0.0
82255,295514,0.0


In [67]:
merged_df = pd.merge(ml_dataset, df, on='student_id', how='left')
merged_df

,student_id,days_engaged,minutes_watched,engaged_with_quizzes,engaged_with_exams,engaged_with_qa,paid,last_active
0,258798,1.0,0.3,False,False,False,False,292.0
1,258799,0.0,0.0,False,False,False,False,292.0
2,258800,29.0,531.2,True,True,False,True,114.0
3,258801,0.0,0.0,False,False,False,False,292.0
4,258802,0.0,0.0,False,False,False,False,292.0
...,...,...,...,...,...,...,...,...
35118,295511,1.0,7.8,True,False,False,False,0.0
35119,295512,1.0,10.4,True,False,False,False,0.0
35120,295513,1.0,0.1,False,False,False,False,0.0
35121,295514,0.0,0.0,False,False,False,False,0.0


In [74]:
merged_df = merged_df.drop('student_id', axis=1)

merged_df.corrwith(merged_df.paid).sort_values(ascending=False)

paid                    1.000000
days_engaged            0.613758
minutes_watched         0.548005
engaged_with_exams      0.525622
engaged_with_quizzes    0.408455
engaged_with_qa         0.254452
last_active            -0.163009
dtype: float64

In [71]:
merged_df.drop('student_id', axis=1)

,days_engaged,minutes_watched,engaged_with_quizzes,engaged_with_exams,engaged_with_qa,paid,last_active
0,1.0,0.3,False,False,False,False,292.0
1,0.0,0.0,False,False,False,False,292.0
2,29.0,531.2,True,True,False,True,114.0
3,0.0,0.0,False,False,False,False,292.0
4,0.0,0.0,False,False,False,False,292.0
...,...,...,...,...,...,...,...
35118,1.0,7.8,True,False,False,False,0.0
35119,1.0,10.4,True,False,False,False,0.0
35120,1.0,0.1,False,False,False,False,0.0
35121,0.0,0.0,False,False,False,False,0.0


In [ ]:
# Add a 'days_passed_since_registration' column